# DeepLabCut Toolbox
https://github.com/AlexEMG/DeepLabCut

Nath\*, Mathis\* et al. *Using DeepLabCut for markerless pose estimation during behavior across species*, (under revision).

This notebook demonstrates the necessary steps to use DeepLabCut for your own project.
This shows the most simple code to do so, but many of the functions have additional features, so please check out the overview & the protocol paper!

This notebook illustrates how to:
- create a project
- extract training frames
- label the frames
- plot the labeled images
- create a training set
- train a network
- evaluate a network
- analyze a novel video
- create an automatically labeled video 
- plot the trajectories

*Note*: Refine a network based after the network was trained on just a few labeled images is illustrated in "Demo-labeledexample-MouseReaching.ipynb". This demo also contains an already labeled data set and is perhaps the best starting point for brand new users.

## Create a new project

It is always good idea to keep the projects seperate. This function creates a new project with subdirectories and a basic configuration file in the user defined directory otherwise the project is created in the current working directory.

You can always add new videos to the project at any stage of the project. 

In [1]:
import deeplabcut

In [2]:
import os
print(os.getcwd())
import matplotlib
%matplotlib inline

/home/morales/marcoabrate/LiftFly3D/prism


In [3]:
'''
task='sideJointTracking' # Enter the name of your experiment Task
experimenter='PrismData' # Enter the name of the experimenter
data_dirs = [
    "/ramdya-nas/SG/prism_data/191125_PR/Fly1/001_prism/behData/images/side_view_191125_PR_Fly1_001_prism/"+\
    "side_video_191125_PR_Fly1_001_prism.mp4",
    "/ramdya-nas/SG/prism_data/191125_PR/Fly1/002_prism/behData/images/side_view_191125_PR_Fly1_002_prism/"+\
    "side_video_191125_PR_Fly1_002_prism.mp4",
    "/ramdya-nas/SG/prism_data/191125_PR/Fly1/003_prism/behData/images/side_view_191125_PR_Fly1_003_prism/"+\
    "side_video_191125_PR_Fly1_003_prism.mp4",
    "/ramdya-nas/SG/prism_data/191125_PR/Fly1/004_prism/behData/images/side_view_191125_PR_Fly1_004_prism/"+\
    "side_video_191125_PR_Fly1_004_prism.mp4"
]
deeplabcut.create_new_project(task, experimenter, data_dirs)
'''

deeplabcut_dir = "/home/morales/marcoabrate/LiftFly3D/prism/sideJointTracking-PrismData-2019-12-17/"
config_path = deeplabcut_dir + "config.yaml"

## Extract frames from videos 
A key point for a successful feature detector is to select diverse frames, which are typical for the behavior you study that should be labeled.

This function selects N frames either uniformly sampled from a particular video (or folder) (algo=='uniform'). Note: this might not yield diverse frames, if the behavior is sparsely distributed (consider using kmeans), and/or select frames manually etc.

Also make sure to get select data from different (behavioral) sessions and different animals if those vary substantially (to train an invariant feature detector).

Individual images should not be too big (i.e. < 850 x 850 pixel). Although this can be taken care of later as well, it is advisable to crop the frames, to remove unnecessary parts of the frame as much as possible.

Always check the output of cropping. If you are happy with the results proceed to labeling.

In [ ]:
'''
deeplabcut.extract_frames(config_path, 'automatic', 'kmeans', crop=False, userfeedback=True)
# there are other ways to grab frames, such as by clustering 'kmeans'; please see the paper.
'''

## Label the extracted frames
Only videos in the config file can be used to extract the frames. Extracted labels for each video are stored in the project directory under the subdirectory **'labeled-data'**. Each subdirectory is named after the name of the video. The toolbox has a labeling toolbox which could be used for labeling. 

In [4]:
#'''
%gui wx
deeplabcut.label_frames(config_path)
#'''

You can now check the labels, using 'check_labels' before proceeding. Then, you can use the function 'create_training_dataset' to create the training dataset.


## Create a training dataset
This function generates the training data information for DeepCut (which requires a mat file) based on the pandas dataframes that hold label information. The user can set the fraction of the training set size (from all labeled image in the hd5 file) in the config.yaml file. While creating the dataset, the user can create multiple shuffles. 

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**.

Now it is the time to start training the network!

In [ ]:
'''
deeplabcut.create_training_dataset(config_path)
'''

## Start training

This function trains the network for a specific shuffle of the training dataset. 

In [4]:
import tensorflow as tf
tf.test.is_gpu_available(
    cuda_only=False,
    min_cuda_compute_capability=None
)

InternalError: failed initializing StreamExecutor for CUDA device ordinal 0: Internal: failed call to cuDevicePrimaryCtxRetain: CUDA_ERROR_OUT_OF_MEMORY; total memory reported: 8370061312

In [ ]:
'''
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)

sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

deeplabcut.train_network(config_path,
                         displayiters=2000,
                         saveiters=20000,
                         maxiters=200000)
'''

## Start evaluating
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [4]:
deeplabcut.evaluate_network(config_path)

/home/morales/marcoabrate/LiftFly3D/prism/sideJointTracking-PrismData-2019-12-17/evaluation-results/  already exists!
/home/morales/marcoabrate/LiftFly3D/prism/sideJointTracking-PrismData-2019-12-17/evaluation-results/iteration-0/sideJointTrackingDec17-trainset95shuffle1  already exists!
Running  DeepCut_resnet50_sideJointTrackingDec17shuffle1_200000  with # of trainingiterations: 200000
INFO:tensorflow:Restoring parameters from /home/morales/marcoabrate/LiftFly3D/prism/sideJointTracking-PrismData-2019-12-17/dlc-models/iteration-0/sideJointTrackingDec17-trainset95shuffle1/train/snapshot-200000


INFO:tensorflow:Restoring parameters from /home/morales/marcoabrate/LiftFly3D/prism/sideJointTracking-PrismData-2019-12-17/dlc-models/iteration-0/sideJointTrackingDec17-trainset95shuffle1/train/snapshot-200000
0it [00:00, ?it/s]

Analyzing data...


480it [00:10, 47.22it/s]


Done and results stored for snapshot:  snapshot-200000
Results for 200000  training iterations: 95 1 train error: 1.65 pixels. Test error: 8.65  pixels.
With pcutoff of 0.1  train error: 1.65 pixels. Test error: 7.76 pixels
Thereby, the errors are given by the average distances between the labels by DLC and the scorer.
The network is evaluated and the results are stored in the subdirectory 'evaluation_results'.
If it generalizes well, choose the best model for prediction and update the config file with the appropriate index for the 'snapshotindex'.
Use the function 'analyze_video' to make predictions on new videos.
Otherwise consider retraining the network (see DeepLabCut workflow Fig 2)


## Start Analyzing videos
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [5]:
img_dirs =[\
"/ramdya-nas/SG/prism_data/191125_PR/Fly1/001_prism/behData/images/side_view_191125_PR_Fly1_001_prism",
"/ramdya-nas/SG/prism_data/191125_PR/Fly1/002_prism/behData/images/side_view_191125_PR_Fly1_002_prism",
"/ramdya-nas/SG/prism_data/191125_PR/Fly1/003_prism/behData/images/side_view_191125_PR_Fly1_003_prism",
"/ramdya-nas/SG/prism_data/191125_PR/Fly1/004_prism/behData/images/side_view_191125_PR_Fly1_004_prism",
"/ramdya-nas/SG/prism_data/191125_PR/Fly2/001_prism/behData/images/side_view_191125_PR_Fly2_001_prism",
"/ramdya-nas/SG/prism_data/191125_PR/Fly2/002_prism/behData/images/side_view_191125_PR_Fly2_002_prism",
"/ramdya-nas/SG/prism_data/191125_PR/Fly2/003_prism/behData/images/side_view_191125_PR_Fly2_003_prism",
"/ramdya-nas/SG/prism_data/191125_PR/Fly2/004_prism/behData/images/side_view_191125_PR_Fly2_004_prism",
          ]

In [7]:
for img_dir in img_dirs[:1]:
    deeplabcut.analyze_time_lapse_frames(config_path,
                                         directory=img_dir,
                                         frametype=".jpg",
                                         rgb=False,
                                         save_as_csv=True)

Using snapshot-200000 for model /home/morales/marcoabrate/LiftFly3D/prism/sideJointTracking-PrismData-2019-12-17/dlc-models/iteration-0/sideJointTrackingDec17-trainset95shuffle1
INFO:tensorflow:Restoring parameters from /home/morales/marcoabrate/LiftFly3D/prism/sideJointTracking-PrismData-2019-12-17/dlc-models/iteration-0/sideJointTrackingDec17-trainset95shuffle1/train/snapshot-200000


INFO:tensorflow:Restoring parameters from /home/morales/marcoabrate/LiftFly3D/prism/sideJointTracking-PrismData-2019-12-17/dlc-models/iteration-0/sideJointTrackingDec17-trainset95shuffle1/train/snapshot-200000


Analyzing all frames in the directory:  /ramdya-nas/SG/prism_data/191125_PR/Fly1/001_prism/behData/images/side_view_191125_PR_Fly1_001_prism


  0%|          | 0/11561 [00:00<?, ?it/s]

Starting to extract posture
Overall # of frames:  11561  found with (before cropping) frame dimensions:  550 260


11615it [05:38, 36.22it/s]                           


Saving results in /ramdya-nas/SG/prism_data/191125_PR/Fly1/001_prism/behData/images/side_view_191125_PR_Fly1_001_prism...
Saving csv poses!
The folder was analyzed. Now your research can truly start!
If the tracking is not satisfactory for some frome, consider expanding the training set.


## Extract outlier frames [optional step]
This is an optional step and is used only when the evaluation results are poor i.e. the labels are incorrectly predicted. In such a case, the user can use the following function to extract frames where the labels are incorrectly predicted. Make sure to provide the correct value of the "iterations" as it will be used to create the unique directory where the extracted frames will be saved.

In [ ]:
path_config_file = '/home/morales/Documents/Morales/fly_lane-DM-2019-02-19/config.yaml'
crop_video = '/home/morales/Documents/Morales/MDN3_Screen/GFP/190315/161441_s0a0_p75/output_40fps_5.avi'

In [ ]:
path_config_file = '/home/morales/Documents/Morales/fly_lane-DM-2019-02-19/config.yaml'
crop_video = '/home/morales/Documents/Morales/MDN3_Screen/GFP/190315/161441_s0a0_p75/output_40fps_5.avi'

deeplabcut.extract_outlier_frames(path_config_file,[crop_video])

## Refine Labels [optional step]
Following the extraction of outlier frames, the user can use the following function to move the predicted labels to the correct location. Thus augmenting the training dataset.

In [ ]:
%gui wx
deeplabcut.refine_labels(path_config_file)

In [ ]:
#Once all folders are relabeled, check them and advance. See how to check labels, above!
deeplabcut.merge_datasets(path_config_file)

## Create a new iteration of training dataset [optional step]
Following the refine labels, append these frames to the original dataset to create a new iteration of training dataset.

In [ ]:
deeplabcut.create_training_dataset(path_config_file)

## Create labeled video
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [ ]:
# Modified
videofile_path = [crop_video] #Enter the list of videos to analyze.

In [ ]:
deeplabcut.create_labeled_video(path_config_file,videofile_path)

## Plot the trajectories of the analyzed videos
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
%matplotlib notebook 
deeplabcut.plot_trajectories(path_config_file,videofile_path)

In [ ]:
sess.close()